## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제없음

In [31]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url = url)

    text = page.read().decode('utf-8')              # print(text)결과 index.html.py
    # 문자열 fine() : text.find(찾을 문자, 시작index, 끝 index) return은 해당위치의 index
    text = text[text.find(f'>{city}</a>'):]       

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]           # <td> 7번째로 이동. 결과: td>구름많음....

    start = 3
    end = text.find('</td>')
    current_temp = text[start:end]      # 인텍스3('td>' 다음 글자)부터 </td>까지 
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도은 {current_humid}% 입니다')


get_weather('부산')



부산의 현재 기온은 11.6˚C 입니다
부산의 현재 습도은 26% 입니다


### OpenAPI 크롤링
- 공공데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스 정보 안내기 현황

```python
response = requests.get(total_url, verify=False)
```

- 문제점
    - https를 일반 request로 부르면 SSL 오류발생
    - 해결하려면 외부모듈 REQUESTS를 사용, verify=False(검증안함) 옵션을 지정 
    - 아래 셀의 19번 라인!


In [48]:
from urllib.parse import quote, unquote, urlencode  # 한글을 URLencode 변환하는 함수
import requests
import json

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = '?' + urlencode(
        {
            'serviceKey': 'Hp7RL4tCw0cXBMTYsWCTrydbix/qtqe4+u5yRNze4LKbniVQhVKmNWMk8IxYObz6/EB41Vo47zCdEVUVRfAvsA==',
            'pageNo': '1',
            'numOfRows':'10',
            'resultType': type,
            'stationLoc': stationName
        }
    )

    total_url = api_url + queryString
    response = requests.get(total_url, verify=False)
    return response.text

try:
    result = getDataPortalSearch('부경대', 'json')
    json_data = json.loads(result)
    
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')

{'stationNum': '07006', 'stationLoc': '경성대.부경대역', 'lat': '35.13856124', 'lng': '129.1023074', 'addr': '남구 대연3동 93-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07007', 'stationLoc': '경성대.부경대역', 'lat': '35.13837971', 'lng': '129.1024864', 'addr': '남구 대연3동 90-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07011', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13287689', 'lng': '129.1012804', 'addr': '남구 대연3동 531-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07012', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13234515', 'lng': '129.1010586', 'addr': '남구 대연3동 537-1', 'insYear': '', 'dataDay': '2023-02-03'}


c:\Source\studyPython\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>' + item['stationLoc'] + '</h4>'+ item['stationNum'] + '<br>' + item['addr']
iframe = folium.IFrame(stop_str)
```
- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}와 호환안됨
    - 구식방법인 문자열 결합방식으로 해결해야함.


In [49]:
import folium

if len(station_data) > 0:   # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location=[center_lat, center_lng], zoom_start = 13)

# 전체 정류소 위치값 마커 표시
for  item in station_data:
    stop_str = '<h4>' + item['stationLoc'] + '</h4>'+ item['stationNum'] + '<br>' + item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width = 200, max_width=200) # 팝업 사이즈 지정
    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                icon=folium.Icon(icon='pushpin')).add_to(m)

m
